In [143]:
import numpy as np
import pandas as pd
from scipy.integrate import quad
from scipy.optimize import minimize
from scipy.stats import norm
from scipy.misc import derivative


# Define constants
H_0 = 70  # km/s/Mpc
c = 2.99792458e5  # km/s
M = -19.3146267582  # M
M_n = M - 5*np.log10(H_0) + 25 #Script_M
Omega_R = 1e-8

In [152]:
# supernova data
data = pd.read_csv('SCPUnion2.1_mu_vs_z.txt', sep='\t', comment='#', names=['z', 'mu', 'mu_err'])
# Load the covariance matrix
data_cov = pd.read_csv('SCPUnion2.1_covmat_nosys.txt', sep='\s+', header=None)

In [153]:
#Calculation of r(z)

def integrand(z, Omega_M, Omega_DE, w):
    return 1 / np.sqrt(Omega_M * (1 + z)**3 + Omega_DE * (1 + z)**(3 * (1 + w)) + Omega_R * (1 + z)**(4))

def r(z, Omega_M, Omega_DE, w):
    return c / H_0 * quad(integrand, 0, z, args=(Omega_M, Omega_DE, w))[0]

In [154]:
#calculation of d_l

def d_L(z, Omega_M, Omega_DE, w):
    return (1 + z) * r(z, Omega_M, Omega_DE, w)

In [155]:
#calculation of m(z, {pj})

def m_th(z, Omega_M, Omega_DE, w):
    return 5 * np.log10(H_0*d_L(z, Omega_M, Omega_DE, w)) + M_n

In [157]:
# Define your chi-squared function
def chi_squared(params):
    Omega_M, Omega_DE, w = params
    m_theoretical = data['z'].apply(lambda z: m_th(z, Omega_M, Omega_DE, w))  # renamed m_th to m_theoretical
    chi_sq = np.sum(((data['mu'] - m_theoretical) / data['mu_err'])**2)  # use m_theoretical here
    if np.isinf(chi_sq):
        print(f"chi_squared returned np.inf for params {params}")
    return chi_sq

# Initial guess for Omega_M, Omega_DE, w
initial_guess = [0.3, 0.7, -1]

# Use scipy's minimize function to find the best-fit parameters
result = minimize(chi_squared, initial_guess)

# Now you can calculate the theoretical distance modulus with the best-fit parameters
data['m_th'] = data['z'].apply(lambda z: m_th(z, result.x[0], result.x[1], result.x[2]))

chi_squared returned np.inf for params [ 0.3  0.7 -1. ]
chi_squared returned np.inf for params [ 0.30000001  0.7        -1.        ]
chi_squared returned np.inf for params [ 0.3         0.70000001 -1.        ]


/opt/anaconda3/lib/python3.11/site-packages/scipy/optimize/_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


chi_squared returned np.inf for params [ 0.3         0.7        -0.99999999]


In [123]:
print(f"Best-fit parameters: Omega_M = {result.x[0]}, Omega_DE = {result.x[1]}, w = {result.x[2]}")

Best-fit parameters: Omega_M = 0.3, Omega_DE = 0.7, w = -1.0


In [96]:
print(data.head())

               z         mu    mu_err       m_th
1993ah  0.028488  35.335551  0.226144  35.963629
1993ag  0.050043  36.675442  0.167114  37.191382
1993o   0.052926  36.816881  0.155866  37.313568
1993b   0.070086  37.440321  0.158669  37.926593
1992bs  0.062668  37.480331  0.156270  37.682299
